<a href="https://colab.research.google.com/github/davide1536/minesweeper/blob/main/mineSweeper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [143]:
import random 
import numpy as np
from pysat.solvers import Solver
from pysat.card import *
from pysat.formula import IDPool
import sys

In [144]:
def checkForVictory(board, knowledge):
  for row in range(SIZE):
    for col in range(SIZE):
      if board[codeId(row, col)].value == -1:
        if board[codeId(row, col)].flag == 0 or board[codeId(row,col)].safe == 1:
          print("trovata falsa mina")
          print(board[codeId(row, col)].id)
          print(board[codeId(row, col)].flag)
          print(board[codeId(row,col)].safe)
          return False
      else:
        if board[codeId(row, col)].flag == 1 or board[codeId(row,col)].safe == 0:
          print("trovata falsa safe")
          print(board[codeId(row, col)].id)
          print(board[codeId(row, col)].flag)
          print(board[codeId(row,col)].safe)
          return False
    return True

In [145]:
def intersection(lst1, lst2):
  return list(set(lst1).intersection(lst2))

In [146]:
def difference(list1, list2):
  return list(set(list1) - set(list2))

In [147]:
def selectAlternativeCell(board, knowledge, tried, numberModels, propositions):
  max = 0
  maxCell = -1
  for cell in tried:
    if board[cell].selected == 0:
      print("seleziono la cella con più probabilità che sia safe")
      print("la cella ",cell, " se è safe ha ",numberModels[cell], " modelli")
      if numberModels[cell] > max:
        maxCell = cell
        max= numberModels[cell]
  
  if not(maxCell == -1):
   return maxCell
  
  for row in range(SIZE):
      for col in range(SIZE):
        if not(board[codeId(row,col)].id in cellSelected) and not(board[codeId(row,col)].id in mineDiscovered):
          return board[codeId(row,col)].id

In [148]:
def selectCellInKnowledge(board, knowledge, tried):
  #print("cerco cella")
  for cells in knowledge.totalCells:
    for cell in cells:
      if board[cell].safe != 1 and board[cell].flag != 1 and not(cell in tried):
        #print(tried)
        #print(cell)
        #print(board[cell].safe)
        #print(board[cell].flag)
        return cell

  

In [149]:
def createClauses(board, knowledge):
  clauses = []
  for i,cells in enumerate(knowledge.totalCells):
    if knowledge.count[i] != 0 and knowledge.count[i] != len(cells) and len(cells) > 0:
      #print("transformo in clausola celle: ", cells)
      #print("count: ", knowledge.count[i])
      pool = IDPool()
      pool.occupy(start=1,stop=81+i*50)

      cnfClause = CardEnc.equals(lits = cells, bound = int(knowledge.count[i]),vpool=pool, encoding=1)
      clauses.append(cnfClause)

  return clauses


In [150]:
def createAssumptions(board,knowledge):
  assumptions = []
  for cell in cellSelected:
    assumptions.append(board[cell].proposition)

  for mine in mineDiscovered:
    assumptions.append(board[mine].proposition)

  return assumptions

In [151]:
def removePositiveProposition(knowledge, currentCells, currentIndex, dictionary):
  for i,cells in enumerate(knowledge.totalCells):
    if i != currentIndex:
      removeAndUpdateProposition(knowledge, currentCells, cells, dictionary, i)
      #print("differenza: ", knowledge.totalCells[i])
      if knowledge.totalCells[i] == []:
        #print("tolgo liste vuote")
        knowledge.count[i] = -1
        # del knowledge.totalCells[i]
        # del knowledge.count[i]

In [152]:
def removeNegativeProposition(knowledge, currentCells, currentIndex, dictionary):
  for i,cells in enumerate(knowledge.totalCells):
    if i != currentIndex:
      numberOfCells = removeAndUpdateProposition(knowledge, currentCells, cells, dictionary,i)
      #print("differenza: ", knowledge.totalCells[i])
      #print("counter attuale: ",knowledge.count[i])
      knowledge.count[i] -= numberOfCells
      #print("counter aggiornato: ", knowledge.count[i])
      if knowledge.totalCells[i] == []:
       # print("tolgo lister vuote")
       knowledge.count[i] = -1
        # del knowledge.totalCells[i]
        # del knowledge.count[i]

In [153]:
def removeAndUpdateProposition(knowledge, currentCells, cells, dictionary, index):

  #print("rimuovo proposizioni")
  intersect= intersection(cells, currentCells)
  #print("intersezione tra ", cells, " e ",currentCells, " é ", intersect)
  diff = difference(knowledge.totalCells[index], intersect)
  knowledge.totalCells[index] = diff
  return len(intersect)
 

In [154]:
def clearKnowledge(knowledge, board):
  for i, cells in enumerate(knowledge.totalCells):
    if cells == [] and knowledge.count[i] == -1:
      del knowledge.totalCells[i]
      del knowledge.count[i]

In [155]:
def updateKnowledge(knowledge,board):

  for i,cells in enumerate(knowledge.totalCells):
      if knowledge.count[i] == 0:
        #print("trovato celle con counter 0")
        for cell in cells:
          # if not(cell in cellSelected):
          #   cellSelected.append(cell)

          if board[cell].proposition > 0:
            board[cell].proposition = -board[cell].proposition
        removePositiveProposition(knowledge, cells, i, board)

  for i,cells in enumerate(knowledge.totalCells):  
      if knowledge.count[i] == len(cells):
        for cell in cells:
          if not(cell in mineDiscovered):
            mineDiscovered.append(cell)
          board[cell].safe = 0
          board[cell].flag = 1
          
        removeNegativeProposition(knowledge, cells, i,board)

In [156]:
def buildCellDictionary(board):
  dictionary = {}
  for row in range(SIZE):
    for col in range(SIZE):
      dictionary[codeId(row, col)] = createCell(board[row,col], row, col)
  
  return dictionary


In [157]:
def codeId(value1, value2):
  return value1 * SIZE + value2 + 1 

In [158]:
def decreasePropositionMagnitude(board,formulas, assumptions):
  propositionSet = []

  for proposition in assumptions:
    propositionSet.append(abs(proposition))
  for formula in formulas:
    for clause in formula.clauses:
      for proposition in clause:
        propositionSet.append(abs(proposition))


  propositionSet = list(dict.fromkeys(propositionSet))
  propositionSet.sort()

  print(propositionSet)

  for i,proposition in enumerate(assumptions):
    index = propositionSet.index(abs(proposition))
    if assumptions[i] > 0:
      assumptions[i] = index + 1
    else:
      assumptions[i] = -(index + 1)

  for formula in formulas:
    for clause in formula.clauses:
      for i,proposition in enumerate(clause):
        index = propositionSet.index(abs(proposition))
        if clause[i] > 0:
          clause[i] = index + 1
        else:
          clause[i] = -(index + 1)


  return propositionSet

  



In [230]:
def findSafeCell(board, knowledge, cellTried, mineNModels):
  # print("knowledge accumulata sino ad ora")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])

 
  
  assumptions = createAssumptions(board, knowledge)
  print("assu: ",assumptions)
  print("len ",len(assumptions))
  if (len(assumptions) == SIZE * SIZE):
    return -4
  setOfFormulas = createClauses(board, knowledge)

  # print("CNF non codificata: ")
  # print("")
  # for formula in setOfFormulas:
  #   print("clauses: ", formula.clauses)
  #   for clause in formula.clauses:
  #     print("clause: ",clause)


  propositions = decreasePropositionMagnitude(board,setOfFormulas, assumptions)
  negativeSolution = ''
  positiveSolution = ''
  cell = 0
  

        
  # print("CNF codificata:")
  # print("")
  # for formula in setOfFormulas:
  #   print("clauses: ", formula.clauses)
  #   for clause in formula.clauses:
  #     print("clause: ",clause)

 

  while not(positiveSolution == False) and not(negativeSolution == False) and not(cell == None):


  
    cell = selectCellInKnowledge(board,knowledge, cellTried)
    if cell == None:
      print("non so che cella scegliere con certezza")
  
      cell = selectAlternativeCell(board,knowledge, cellTried, mineNModels, propositions)
  
      return cell
    if not(cell == None):
      cellTried.append(cell)


      
      print("provo a selezionare la cella: ", cell)
      
      if board[cell].proposition > 0:
        goalProp = propositions.index(abs(cell)) + 1
      else:
        goalProp = -(propositions.index(abs(cell)) + 1)

      #print("con proposizione: ", board[cell].proposition)
     

          

      
      positiveSolver = Solver()
      negativeSolver = Solver()
    
      for formula in setOfFormulas:
        
        for clause in formula.clauses:
          positiveSolver.add_clause(clause)
          negativeSolver.add_clause(clause)

  
      #positiveSolver.append_formula(formula=clauses)
      #print("proposition: ", int(-board[cell].proposition))
      negativeSolver.add_clause([goalProp])

      
      #negativeSolver.append_formula(clauses, no_return=True)
      positiveSolver.add_clause([-goalProp])

      negativeSolution = negativeSolver.solve(assumptions = assumptions)
      positiveSolution = positiveSolver.solve(assumptions = assumptions)

      enumerating = (positiveSolver.enum_models(assumptions = assumptions))

    
      # n_models = 0
      # for m in enumerating:
      #   print(m)
      #   n_models += 1

      
      n_models = len(list(enumerating)) 

      
      print("modelli: ", n_models)

      mineNModels[cell] = n_models

      print(negativeSolution)
      print(positiveSolution)

      negativeSolver.delete()
      positiveSolver.delete()

  if negativeSolution == False:
    print("ho trovato una cella safe ",cell)
    return cell
  elif positiveSolution == False:
    print("ho trovato una mina")
    createKnowledge(board[cell], board, knowledge)
    return -2


In [160]:
def createKnowledge(cell, board, knowledge):
  #print("creo knowledge della cella in posizione: ", cell.row, " ", cell.col)
  currentKnowledge = []
  if cell.value == -1:
    currentKnowledge.append(cell.id)
    knowledge.count.append(1)
  else:

    currentKnowledge.append(cell.id)
    knowledge.totalCells.append(currentKnowledge)
    knowledge.count.append(0)

    currentKnowledge = []
    
    value = cell.value
    for i in range(3):
      for j in range(3):
        if (cell.row-1+i >= 0 and cell.row-1+i <= SIZE-1) and (cell.col-1+j >= 0 and cell.col-1+j <= SIZE-1):
          if cell.id != board[codeId(cell.row-1+i,cell.col-1+j)].id:
              
              currentKnowledge.append(board[codeId(cell.row-1+i,cell.col-1+j)].id)
  
    knowledge.count.append(value)

  knowledge.totalCells.append(currentKnowledge)
  # print("knowledge accumulata sino ad ora")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])
  updateKnowledge(knowledge,board)
  clearKnowledge(knowledge, board)

  # print("knowledge aggiornata")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])
  # print("dopo l'aggiornamento: ")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])


In [161]:
def checkTrivialSafeCell(board, knowledge):
  for i,cells in enumerate(knowledge.totalCells):
    if knowledge.count[i] == 0:
      
      for cell in cells:
        if board[cell].selected == 0:
          return cell
  
  return -1

In [227]:
def selectNextSafeCell(board, knowledge):
  inferenceCell = []
  dictionary = {}
  nextCell = checkTrivialSafeCell(board, knowledge)
  #if there aren't any trivial safe cell use sat solver on clause

  while nextCell == -1:
    print("provo trovare cella con inferenza")
    nextCell = findSafeCell(board,knowledge, inferenceCell, dictionary)
    
    if nextCell == -2:
      nextCell = checkTrivialSafeCell(board,knowledge)

  if nextCell == -4:
    return -4
 
  # if nextCell == -1:
  #   nextCell = findSafeCell(board, knowledge)

  # #if i've found a mine thanks to the solver i'll try to reuse the trivial solver
  # if nextCell == -2:
  #   nextCell = checkTrivialSafeCell(board, knowledge)

  
 
  result = board[nextCell]
    
  return result


In [163]:
def countMines(row,col,board, size):
  counter = 0

  for i in range(3):
    for j in range(3):
      if (row-1+i >= 0 and row-1+i <= size-1) and (col-1+j >= 0 and col-1+j <= size-1):
        if board[row-1+i, col-1+j] == -1:
          counter += 1
        
     
  
  return counter

In [164]:
#function that, for each cell, counts the number of mines around it
def fillBoard(board, size):
  
  for row in range(size):
    for col in range(size):
      if board[row,col] != -1:
        n_mines = countMines(row, col, board, size)
        board[row,col] = n_mines
    
      
    
  return board


In [165]:

def buildBoard(size, probability):
  total_mines = 0
  #board[i][j] = 0 --> no bomb
  #board[i][j] = 1 --> bomb
  print("inizializzo board di gioco")
  board = np.zeros((size,size))
  for row in range(size):
    for col in range(size):
      randomNumber = random.uniform(0,1)
      if randomNumber <= probability:
        board[row,col] = -1
        total_mines += 1
      else:
        board[row,col] = 0


  board = fillBoard(board, size)
  print("board di gioco inizializzata con ",total_mines, " ","mine")
  return board
      

In [166]:
def createCell(value, row, col):
  cell = Cell()
  cell.value = int(value)
  
  cell.safe = 0
    
  cell.selected = 0
  cell.flag = 0
  cell.row = row
  cell.col = col
  cell.proposition = int(codeId(row,col)) 
  cell.id = int(codeId(row,col)) 
  return cell

In [167]:
def startGame(board, knowledge):
  result = ""
  firstCell = board[codeId(0,0)]
  firstCell.selected = 1
  firstCell.safe = 1
  cellSelected.append(firstCell.id)

  if firstCell.value == -1:
    result = "lose"
  else:
    createKnowledge(firstCell, board, knowledge)

 

  while not(result == "win") and not(result == "lose"):
    
    cell = selectNextSafeCell(board, knowledge)
   
    # if cell == -1:
    #   return "could not find a safe cell"
    
    if (len(cellSelected) + len(mineDiscovered) == SIZE * SIZE) or cell == -4:
      victory = checkForVictory(board, knowledge)
      if victory:
        result = "win"
      else:
        return "An error occured"
    else:
      print("ho selezionato la cella ", cell.id)
      cell.selected = 1
      cell.safe = 1
      if not(cell.id in cellSelected):
        cellSelected.append(cell.id)
     

      if cell.value == -1:
        result = "lose"
      else:
        createKnowledge(cell, board, knowledge)


  return result

In [168]:
class InternalRepresentation:
  def __init__(self) -> None:
    self.totalCells = []
    self.count = []

In [169]:
class Cell:
  def __init__(self) -> None:
    self.safe = 0
    self.value = 0
    self.proposition = 0
    self.id = 0
    self.selected = 0
    self.row = 0
    self.col = 0
    self.flag = 0

In [233]:
#build a 9x9 board
SIZE = 9
bomb_probability = 1/5
board = buildBoard(SIZE, bomb_probability)
print(board)

inizializzo board di gioco
board di gioco inizializzata con  18   mine
[[ 0.  0.  0.  0.  1. -1. -1.  2.  0.]
 [ 0.  1.  1.  1.  1.  3. -1.  2.  0.]
 [ 1.  2. -1.  2.  1.  2.  1.  2.  1.]
 [-1.  2.  2.  3. -1.  3.  3.  3. -1.]
 [ 1.  1.  1. -1.  3. -1. -1. -1.  2.]
 [ 1.  1.  1.  2.  3.  3.  3.  2.  1.]
 [-1.  3.  1.  2. -1.  1.  0.  0.  0.]
 [-1.  5. -1.  2.  1.  2.  1.  1.  0.]
 [-1. -1.  2.  1.  0.  1. -1.  1.  0.]]


In [234]:
#start game from the up-left corner
knowledge = InternalRepresentation();
cellSelected = []
mineDiscovered = []
boardDictionary = buildCellDictionary(board)
result = startGame(boardDictionary, knowledge)
print("risultato: ", result);      

ho selezionato la cella  10
ho selezionato la cella  2
ho selezionato la cella  11
ho selezionato la cella  19
ho selezionato la cella  20
ho selezionato la cella  3
ho selezionato la cella  12
ho selezionato la cella  4
ho selezionato la cella  13
ho selezionato la cella  22
ho selezionato la cella  5
ho selezionato la cella  14
ho selezionato la cella  23
provo trovare cella con inferenza
assu:  [-1, -10, -2, -11, -19, -20, -3, -12, -4, -13, -22, -5, -14, -23, 21]
len  15
[1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15, 19, 20, 21, 22, 23, 24, 28, 29, 30, 31, 32, 33, 382, 383, 632, 633, 732, 733, 782, 783, 784, 785]
provo a selezionare la cella:  28
modelli:  2
True
True
provo a selezionare la cella:  29
modelli:  2
True
True
provo a selezionare la cella:  30
modelli:  4
False
True
ho trovato una cella safe  30
ho selezionato la cella  30
provo trovare cella con inferenza
assu:  [-1, -10, -2, -11, -19, -20, -3, -12, -4, -13, -22, -5, -14, -23, -30, 21]
len  16
[1, 2, 3, 4, 5, 6, 10, 11, 12